In [1]:
import os
import boto3
import os.path as op
import diff_classifier.aws as aws
import cloudknot as ck
import diff_classifier.knotlets as kn
import numpy as np

iteration = 23

In [2]:
def print_hello_world(num):
    import diff_classifier.aws as aws
    
    folder = 'test_files'
    test = 'Hello world!'
    filename = 'hello_world_{}.txt'.format(num)
    text_file = open(filename, 'w')
    text_file.write(test)
    text_file.close()
    aws.upload_s3(filename, '{}/{}'.format(folder, filename), bucket_name='hpontes.data')
    print('Successfully uploaded file to s3')

In [3]:
github_installs=('https://github.com/ccurtis7/diff_classifier.git')
my_image = ck.DockerImage(func=print_hello_world, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

docker_file = open(my_image.docker_path)
docker_string = docker_file.read()
docker_file.close()

req = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'))
req_string = req.read()
req.close()

new_req = req_string[0:req_string.find('\n')-4]+'5.28'+ req_string[req_string.find('\n'):]
req_overwrite = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'), 'w')
req_overwrite.write(new_req)
req_overwrite.close()

In [4]:
integers = list(range(0, 11))

knot = ck.Knot(name='hello_world_{}'.format(iteration),
               docker_image = my_image,
               resource_type = 'SPOT',
               bid_percentage = 100,
               #image_id = 'ami-0112e1134421389c0',
               pars_policies=('AmazonS3FullAccess',))
result_futures = knot.map(integers)

In [ ]:
knot.clobber()

In [ ]:
print_hello_world(2)